In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [2]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR.xml")
mod_RPE_PR.id = 'RPE_PR'
mod_Human1_Human1 = read_sbml_model(Path().cwd() / "models" /  "mod_Human1_Human1.xml")
mod = read_sbml_model(Path().cwd() / 'models/Human-GEM.xml')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [3]:
# modify models
# load blood exchange bounds
import pandas as pd
df = pd.read_excel(Path().cwd() / 'rxn_bounds/blood_selection.xlsx') # with col names 'id' 'lb' 'ub'

# set objective function (PR ATP DM)
mod_RPE_PR.objective = 'MAR03964_PR'
mod_Human1_Human1.objective = 'MAR03964_PR'
mod.objective = 'MAR03964'

In [4]:
# modify models (blood exchange)
for m in [mod_RPE_PR, mod_Human1_Human1]:
    m.objective = 'MAR03964_PR' # ATP demand in PR
    m.remove_reactions([r.id for r in  m.reactions if len(r.products)==0 if 'PR' in r.id]) # remove PR exchange reactions
    m.remove_metabolites([m for m in m.metabolites if len(m.reactions) == 0])              # prune metabolites that are unused
    for r in [r for r in m.reactions if len(r.products) == 0 if '_RPE' in r.id]:           # open efflux for RPE exchange reactions
        r.bounds=(0,1000) # close all exchange reactions
    for index, row in df.iterrows():  
        if row.id in [r.id for r in m.reactions]:
            m.reactions.get_by_id(row['id']).bounds = (row['lb'],row['ub'])# allow influx for RPE exchange reactions (blood exchange)

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\core\group.py:147: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [5]:
def fba_analysis(mod, boundary_dicts, boundary_labels=dict(),results_name=''):    
    # inputs: model, list of boundary dicts {rxnID:(lb,ub)}, objective function rxnID (string)
    import pandas as pd
    from datetime import datetime
    from src.get_info import make_rxn_df, make_compact_rxn_df
    from datetime import datetime

    #  create empty dicts
    bounds = dict() # changed model bounds
    conditions = dict()
    ovs = dict() # objective values
    f = dict() # fba fluxes
    uptake = dict() # uptake fluxes
    secretion = dict() # secretion fluxes
        
    #   set counter, i, to 1
    i = 0
        
    with mod as m:
            
        # run analysis for every dict in list
        for d in boundary_dicts:
            for k in d:   # set bounds for all keys (rxnIDs in dict)
                m.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict

            # run analysis
            fba = m.optimize() # fba
            
            # objective values
            ovs[i] = [m.objective, m.objective.direction, fba.objective_value]
            
            if fba.status == 'optimal':

                # write analysis info / results into dicts

                # uptake / secretion
                uptake_summary = m.summary().uptake_flux
                secretion_summary = m.summary().secretion_flux
                uptake[i] = pd.DataFrame(uptake_summary['flux'])
                secretion[i] = pd.DataFrame(secretion_summary['flux'])
                
                # all fluxes
                f[i] = fba.to_frame()['fluxes']
          
            # update counter
            i=i+1
    
    # prepare dfs for excel sheet
    model_info = pd.DataFrame([m.id, m.name,m.compartments,m.annotation],\
             index=['id','name','compartments','annotation'],columns = ['model'])
    
    rxn_df = make_rxn_df(m)   
    compact_rxn_df = make_compact_rxn_df(m)  
    
    met_df = pd.DataFrame([[mi.name,mi.compartment,mi.formula,mi.charge,[r.id for r in list(mi.reactions)]] for mi in m.metabolites],\
             index=[m.id for m in m.metabolites],columns=['name','compartment','formula','charge','reactions'])
    
    ovs_df  = pd.concat([pd.DataFrame(boundary_dicts),\
                         pd.DataFrame(boundary_labels),\
                          pd.DataFrame(ovs, index=['objective','objective_direction','objective_value']).T],axis=1)
    
    # uptake / secretion dfs

    uptake_df = pd.concat(uptake,axis=1)
    secretion_df = pd.concat(secretion,axis=1)
    
    secretion_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                for mets in secretion_df.index],\
                               columns=['metabolite'],index= secretion_df.index)
    uptake_mets = pd.DataFrame([[met.name for met in m.reactions.get_by_id(mets).reactants] \
                                for mets in uptake_df.index],\
                               columns=['metabolite'],index= uptake_df.index)
    
    uptake_df = pd.merge(secretion_mets, uptake_df, left_index=True, right_index=True)  
    secretion_df = pd.merge(secretion_mets, secretion_df, left_index=True, right_index=True)  
                                     
    # sort uptake / secretion dfs
    uptake_df = uptake_df.reindex(uptake_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=False).index)
    secretion_df = secretion_df.reindex(secretion_df.fillna(0).sort_values(by=[c for c in secretion_df.columns if 'flux' in c],ascending=True).index)
    
    # fluxes df
    fluxes_df = pd.concat([pd.DataFrame(f)],keys=["flux"],axis=1)
    fluxes_df = fluxes_df =pd.merge(compact_rxn_df, fluxes_df,left_index=True, right_index=True)
    # sort table on absolute flux size 
    fluxes_df= fluxes_df.reindex(fluxes_df[[c for c in fluxes_df.columns if 'flux' in c]].abs().sort_values(by=[c for c in fluxes_df.columns if 'flux' in c],ascending=False).index)
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
    # select internal fluces only
    fluxes_df[fluxes_df.index.isin([r.id for r in m.reactions if len(r.products)>0])]
        
    # date stamp
    datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')   
    
    # write excel file
    with pd.ExcelWriter('fba_analysis_' + results_name + '_' + datestr + '.xlsx') as writer:  
        model_info.to_excel(writer, sheet_name = 'model')
        rxn_df.to_excel(writer, sheet_name = 'reactions')
        met_df.to_excel(writer, sheet_name = 'metabolites')
        ovs_df.to_excel(writer, sheet_name = 'objective_values')  
        uptake_df.to_excel(writer, sheet_name = 'uptake')    
        secretion_df.to_excel(writer, sheet_name = 'secretion')    
        fluxes_df.to_excel(writer, sheet_name = 'fluxes')     
        
    return [ovs_df, uptake_df, secretion_df, fluxes_df]

In [6]:
blood_ratio = {df['id'][i]:df['ratio [glucose]/[x]'] [i] for i in range(len(df['id']))}
# blood_ratio.update((x, y*glucose_level) for x, y in blood_ratio.items())
blood_ratio

{'MAR09034_RPE': 1.0,
 'MAR09063_RPE': 0.12618432385874245,
 'MAR09061_RPE': 0.090869939707149,
 'MAR09067_RPE': 0.05555555555555555,
 'MAR09041_RPE': 0.050818260120585705,
 'MAR09046_RPE': 0.05017226528854436,
 'MAR09068_RPE': 0.0409130060292851,
 'MAR09044_RPE': 0.03574504737295435,
 'MAR09069_RPE': 0.03229974160206718,
 'MAR09040_RPE': 0.026485788113695088,
 'MAR09038_RPE': 0.023040482342807925,
 'MAR09066_RPE': 0.017226528854435832,
 'MAR09064_RPE': 0.016731266149870805,
 'MAR09043_RPE': 0.01309216192937123,
 'MAR09065_RPE': 0.01119724375538329,
 'MAR09045_RPE': 0.009474590869939707,
 'MAR09071_RPE': 0.009323858742463394,
 'MAR09062_RPE': 0.008828596037898364,
 'MAR09042_RPE': 0.005383290267011197,
 'MAR09070_RPE': 0.0032730404823428077,
 'MAR11961_RPE': 0.00014857881136950905,
 'MAR09039_RPE': 0.014233419465977604}

In [8]:
def open_blood_exchange(mod,br, glucose_level):  
    br_updated = br.copy()
    br_updated.update((x, y * glucose_level) for x, y in br_updated.items())
    for key in br:
        mod.reactions.get_by_id(key).bounds = (br_updated[key],1000)# allow influx for RPE exchange reactions (blood exchange)
        
def close_blood_exchange(mod,br):  
    for key in br:
        mod.reactions.get_by_id(key).bounds = (0,1000)# allow influx for RPE exchange reactions (blood exchange)

In [9]:
from src.analysis import create_permutation_dicts
# MAR09048_RPE = oxygen exchange
# MAR09034_RPE = glucose exchange
# MAR09135_RPE = lactate exchange
# MAR04896_PR_RPE =  O2[c_PR] <=> O2[e_RPE_PR]

dark_old_labels = create_permutation_dicts({'oxygen':['anaerobic','limited oxygen (2.2)','unlimited oxygen'],\
                      'glucose':['no glucose','limited glucose (2.6)','unlimited glucose'],\
                      'lactate':['lactate efflux > 0','lactate efflux > 4.53','unlimited lactate exchange'],\
                      'oxygen RPE -> PR':['fixed (1.9)','unlimited']})
dark_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(0,1000),(-2.2,1000),(-1000,1000)],\
                      'MAR09034_RPE':[(0, 1000),(-2.62,1000),(-1000,1000)],\
                      'MAR09135_RPE':[(0, 1000),(4.53,1000),(-1000,1000)],\
                      'MAR04896_PR_RPE':[(-1.9,-1.9),(-1000,1000)]})

dark_new_labels = create_permutation_dicts({'oxygen':['anaerobic','limited oxygen (3.51)','unlimited oxygen'],\
                      'glucose':['no glucose','limited glucose (4.18)','unlimited glucose'],\
                      'lactate':['lactate efflux > 0','lactate efflux > 7.25','unlimited lactate exchange'],\
                      'oxygen RPE -> PR':['fixed (3.21)','unlimited']})
dark_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(0,1000),(-3.51,1000),(-1000,1000)],\
                      'MAR09034_RPE':[(0, 1000),(-4.18,1000),(-1000,1000)],\
                      'MAR09135_RPE':[(0, 1000),(7.25,1000),(-1000,1000)],\
                      'MAR04896_PR_RPE':[(-3.21,-3.21),(-1000,1000)]})

light_old_labels = create_permutation_dicts({'oxygen':['anaerobic','limited oxygen (1.84)','unlimited oxygen'],\
                      'glucose':['no glucose','limited glucose (1.37)','unlimited glucose'],\
                      'lactate':['lactate efflux > 0','lactate efflux > 2.81','unlimited lactate exchange'],\
                      'oxygen RPE -> PR':['fixed (1.59)','unlimited']})
light_old_estimates = create_permutation_dicts({'MAR09048_RPE':[(0,1000),(-1.84,1000),(-1000,1000)],\
                      'MAR09034_RPE':[(0, 1000),(-1.37,1000),(-1000,1000)],\
                      'MAR09135_RPE':[(0, 1000),(2.81,1000),(-1000,1000)],\
                      'MAR04896_PR_RPE':[(-1.59,-1.59),(-1000,1000)]})

light_new_labels = create_permutation_dicts({'oxygen':['anaerobic','limited oxygen (2.61)','unlimited oxygen'],\
                      'glucose':['no glucose','limited glucose (1.93)','unlimited glucose'],\
                      'lactate':['lactate efflux > 0','lactate efflux > 2.95','unlimited lactate exchange'],\
                      'oxygen RPE -> PR':['fixed (2.31)','unlimited']})
light_new_estimates = create_permutation_dicts({'MAR09048_RPE':[(0,1000),(-2.61,1000),(-1000,1000)],\
                      'MAR09034_RPE':[(0, 1000),(-1.93,1000),(-1000,1000)],\
                      'MAR09135_RPE':[(0, 1000),(2.95,1000),(-1000,1000)],\
                      'MAR04896_PR_RPE':[(-2.31,-2.31),(-1000,1000)]})

OLD

dark	glucose (in)	pmol/s/mm2 	2.62

light	glucose (in)	pmol/s/mm2 	1.37

NEW

dark	glucose (in)	pmol/s/mm2 	4.18

light	glucose (in)	pmol/s/mm2 	1.93

In [ ]:
# mod_RPE_PR
open_blood_exchange(mod_RPE_PR,blood_ratio,-2.62)
dark_old_CS = fba_analysis(mod_RPE_PR, dark_old_estimates, dark_old_labels, 'CS_dark_old_ox2.2')
light_old_CS = fba_analysis(mod_RPE_PR, light_old_estimates, light_old_labels, 'CS_light_old_ox1.84')
open_blood_exchange(mod_RPE_PR,blood_ratio,-4.18)            
dark_new_CS = fba_analysis(mod_RPE_PR, dark_new_estimates, dark_new_labels, 'CS_dark_new_ox3.51')
light_new_CS = fba_analysis(mod_RPE_PR, light_new_estimates, light_new_labels, 'CS_light_new_ox2.61')

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\u

KeyboardInterrupt: 

In [11]:
# mod_Human1_Human1
open_blood_exchange(mod_Human1_Human1,blood_ratio,-2.62)
dark_old_H1 = fba_analysis(mod_Human1_Human1, dark_old_estimates, dark_old_labels, 'H1_dark_old_ox2.2')
light_old_H1 = fba_analysis(mod_Human1_Human1, light_old_estimates, light_old_labels, 'H1_light_old_ox1.84')
open_blood_exchange(mod_Human1_Human1,blood_ratio,-4.18)     
dark_new_H1 = fba_analysis(mod_Human1_Human1, dark_new_estimates, dark_new_labels, 'H1_dark_new_ox3.51')
light_new_H1 = fba_analysis(mod_Human1_Human1, light_new_estimates, light_new_labels, 'H1_light_new_ox2.61')

C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\prins\miniconda3\envs\RPE-PR_pytest\lib\site-packages\cobra\u